# Box Plots of Cluster experiments

## Load Data

In [ ]:
%matplotlib ipympl
%matplotlib widget

import sqlite3
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import IPython.display
import ipywidgets

# database is from metrics-to-sqlite.py with only the environments:
# "reytchison-instance-repeat", "reytchison-sample-hardware", "reytchison-host-repeat"

db = sqlite3.connect("amz_benchmark_data_20241024_repeat_host_repeat_instance_and_sample_hardware.sqlite")

In [ ]:
def _get_pop(
    metric: str, workload: str, task: str, dist_ver: str, environment: str,
) -> list[float]:
    q = f"""
    SELECT runs.run_group, "{metric}".value FROM runs JOIN "{metric}" ON "{metric}".run_id = runs.id
    WHERE
        runs.workload = :workload AND
        runs.distribution_version=:dist_ver AND
        runs.environment=:environment AND
        "{metric}".task=:task AND
        "{metric}".sample_type='normal';
    """
    params = {
        "workload": workload,
        "dist_ver": dist_ver,
        "environment":environment,
        "task": task,
    }
    cur = db.execute(q, params)
    return cur.fetchall()

def _get_workloads() -> dict[str, list[str]]:
    cur = db.execute(
        "SELECT DISTINCT workload, task FROM runs JOIN tasks ON run_id = id"
    )
    res: dict[str, list[str]] = defaultdict(list)
    for workload, task in cur.fetchall():
        res[workload].append(task)
    return res

In [ ]:
# global variables
os_dist = "2.16.0"
es_dist = "8.15.0"
environment_same_cluster = "reytchison-instance-repeat"
environment_different_cluster = "reytchison-sample-hardware"
environment_same_host = "reytchison-host-repeat"
incomplete_tests = {
    "2024_10_21_13_43_01",
    "2024_10_21_05_55_30",
    "2024_10_17_16_54_07",
    "2024_10_17_19_36_44",
}

CPU_8275CL = "Intel 8275CL"
CPU_8124M = "Intel 8124M"
CPUS = [CPU_8275CL, CPU_8124M]
# manually added by checking hw-info_[environment-name].tar.gz files
lookup_test_cpu = {
    # same instance ES tests
    "2024_10_20_23_06_45": CPU_8275CL,
    "2024_10_20_08_29_48": CPU_8275CL,
    "2024_10_19_17_53_54": CPU_8275CL,
    "2024_10_19_03_18_05": CPU_8275CL,
    "2024_10_18_12_41_29": CPU_8275CL,
    "2024_10_17_22_07_52": CPU_8275CL,
    # same instance OS
    "2024_10_20_21_03_10": CPU_8275CL,
    "2024_10_20_12_11_17": CPU_8275CL,
    "2024_10_20_03_18_53": CPU_8275CL,
    "2024_10_19_18_27_14": CPU_8275CL,
    "2024_10_19_09_34_38": CPU_8275CL,
    "2024_10_19_00_42_47": CPU_8275CL,
    "2024_10_18_15_50_12": CPU_8275CL,
    "2024_10_18_06_57_25": CPU_8275CL,
    "2024_10_17_22_10_39": CPU_8275CL,
    # different instance ES
    "2024_10_18_22_25_37": CPU_8275CL,
    "2024_10_18_22_26_14": CPU_8275CL,
    "2024_10_18_22_27_06": CPU_8124M,
    "2024_10_18_22_28_13": CPU_8275CL,
    # different instance OS:
    "2024_10_18_22_23_19": CPU_8275CL,
    "2024_10_18_22_23_59": CPU_8275CL,
    "2024_10_18_22_25_14": CPU_8275CL,
    "2024_10_18_22_24_40": CPU_8275CL,
    # same host ES:
    "2024_10_23_18_29_25": CPU_8124M,
    "2024_10_23_18_29_58": CPU_8124M,
    "2024_10_23_18_27_07": CPU_8124M,
    "2024_10_23_18_28_03": CPU_8124M,
    # same host OS:
    "2024_10_23_18_26_04": CPU_8124M,
    "2024_10_23_18_23_55": CPU_8124M,
    "2024_10_23_18_25_13": CPU_8124M,
    "2024_10_23_18_24_32": CPU_8124M,
    
}
cpu_color_lookup = {
    CPU_8275CL: 'lightblue',
    CPU_8124M: 'lightgreen',
}
version_map = {"2.16.0": "OS", "8.15.0": "ES"}


## Plot

In [ ]:
def preprocess_pop(pop):
    metrics = defaultdict(list)
    for (label, val) in pop:
        # skip incomplete tests
        if label in incomplete_tests:
            continue
        metrics[label].append(val)
    labels = list(metrics.keys())
    labels.sort()
    counts = [len(metrics[label]) for label in labels]
    correct_count = max(counts)
    # make sure the number of samples is correct
    for label, count in zip(labels, counts):
        assert count == correct_count, f"{label} has different count {count} from {correct_count}"
    values = [np.array(metrics[label]) for label in labels]
    return (values, labels)

In [ ]:
def _subplot(pop, ax, plt, legend=False, label_xticks=True):
    values, labels = preprocess_pop(pop)
    bp = ax.boxplot(values, widths=0.5, patch_artist=True, flierprops={'markersize': 4})
    for label, box in zip(labels, bp['boxes']):
        cpu = lookup_test_cpu[label]
        color = cpu_color_lookup[cpu]
        box.set_facecolor(color)
    if label_xticks:
        ax.set_xticklabels(labels, rotation=30, ha='right')
    ax.tick_params(axis='x', labelsize=8)
    if legend:
        legend_elements = [plt.Rectangle((0, 0), 1, 1, fc=color, label=label) 
                   for label, color in cpu_color_lookup.items()]
        ax.legend(title="CPU", handles=legend_elements, loc='upper left', ncol=len(cpu_color_lookup), fontsize="small")

In [ ]:
def _plot_same_vs_diff_instance(metric, task, dist_version):
    workload = "big5"
    engine_type = version_map[dist_version]
    pop_repeat_cluster = _get_pop(metric, workload, task, dist_version, environment_same_cluster)
    pop_different_cluster = _get_pop(metric, workload, task, dist_version, environment_different_cluster)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11,6), sharey=True)
    fig.suptitle(f"{metric}: {engine_type} {task} ({workload})")
    plt.subplots_adjust(left=0.1, bottom=0.2)
    _subplot(pop_repeat_cluster, ax1, plt, legend=True)
    ax1.set_title("Same Instance")
    _subplot(pop_different_cluster, ax2, plt)
    ax2.set_title("Different Instances")

In [ ]:
def _plot_all(metric, task, dist_version):
    workload = "big5"
    engine_type = version_map[dist_version]
    pop_repeat_cluster = _get_pop(metric, workload, task, dist_version, environment_same_cluster)
    pop_different_cluster = _get_pop(metric, workload, task, dist_version, environment_different_cluster)
    pop_repeat_host_cluster = _get_pop(metric, workload, task, dist_version, environment_same_host)
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(11,6), sharey=True)
    fig.suptitle(f"{metric}: {engine_type} {task} ({workload})")
    plt.subplots_adjust(left=0.1, bottom=0.2)
    _subplot(pop_repeat_cluster, ax1, plt, legend=True, label_xticks=True)
    ax1.set_title("Same Instance")
    _subplot(pop_different_cluster, ax2, plt, label_xticks=True)
    ax2.set_title("Different Instances")
    _subplot(pop_repeat_host_cluster, ax3, plt, label_xticks=True)
    ax3.set_title("Same Host (Different Instances)")

In [ ]:
def plot(metric, task, dist_version, plot_type):
    plot_types = {
        "all": _plot_all,
        "same_vs_diff_instance": _plot_same_vs_diff_instance
    }
    if plot_type not in plot_types.keys():
        print(f"ERROR: plot_type must be one of {', '.join(list(plot_types.keys()))}")
        return
    plot_func = plot_types[plot_type]
    plot_func(metric, task, dist_version)

# Plot Results

In [ ]:
# change these variables
metric = "service_time"
task = "range-auto-date-histo"
plot_type = "all" # options: "all", "same_vs_diff_instance"
dist = os_dist # options: os_dist, es_dist

In [ ]:
# plot
plot(metric, task, dist, plot_type)

# Save plots of all results

In [ ]:
from pathlib import Path
workloads = _get_workloads()
tasks = workloads["big5"]
def save_plots_for_all(metric: str, plot_type, output_dir: Path):
    plt.ioff()
    for task in tasks:
        for dist in [es_dist, os_dist]:
            plot(metric, task, dist, plot_type=plot_type)
            engine_type = version_map[dist]
            img_path = output_dir / f"{task}-{engine_type}-{metric}.png"
            plt.savefig(img_path, dpi=300)
            plt.close()
    plt.ion()

In [ ]:
metric = "service_time"
output_dir = Path("cluster_experiment_plots_same_vs_diff_instances")
output_dir.mkdir(exist_ok=True)
plot_type = "same_vs_diff_instance"
save_plots_for_all(metric, plot_type, output_dir)

In [ ]:
metric = "service_time"
output_dir = Path("cluster_experiment_plots_all")
output_dir.mkdir(exist_ok=True)
plot_type = "all"
save_plots_for_all(metric, plot_type, output_dir)

## Calculate Variance Heuristic
Calculates a heuristic for how much the tests vary in each experiment. The heuristic counts the fraction of tests in each experiment which have a median that is not between the mean 1st and 3rd quartiles.

In [ ]:
import matplotlib.cbook as cbook
def _do_calculate_within_quartiles(pop):
    values, _ = preprocess_pop(pop)
    stats = cbook.boxplot_stats(values)
    median_values = [stat['med'] for stat in stats]
    q3_vals = [stat['q3'] for stat in stats]
    q3_mean = np.mean(q3_vals)
    q1_vals = [stat['q1'] for stat in stats]
    q1_mean = np.mean(q1_vals)
    within_avg_quartiles = float(sum(q1_mean <= median <= q3_mean for median in median_values)/len(median_values))
    return round(within_avg_quartiles, 3)

def _calculate_within_quartiles_task(metric, task, dist_version):
    workload = "big5"
    pop_repeat_cluster = _get_pop(metric, workload, task, dist_version, environment_same_cluster)
    pop_different_cluster = _get_pop(metric, workload, task, dist_version, environment_different_cluster)
    pop_repeat_host_cluster = _get_pop(metric, workload, task, dist_version, environment_same_host)
    within_repeat_cluster = _do_calculate_within_quartiles(pop_repeat_cluster)
    within_different_cluster = _do_calculate_within_quartiles(pop_different_cluster)
    within_repeat_host_cluster = _do_calculate_within_quartiles(pop_repeat_host_cluster)
    return [within_repeat_cluster, within_different_cluster, within_repeat_host_cluster]

def calculate_within_quartiles(metric: str, tasks):
    header = ["task", "engine_type", "Same Instance", "Different Instance", "Same Host (Different Instance)"]
    rows = [header]
    # get data
    for task in tasks:
        for dist in [es_dist, os_dist]:
            within = _calculate_within_quartiles_task(metric, task, dist)
            row = [task, version_map[dist]]
            row.extend(within)
            rows.append(row)
    return rows

In [ ]:
import csv
workloads = _get_workloads()
tasks = workloads["big5"]
metric = "service_time"
outfile = "cluster_experiment_tasks_within_mean_quartiles.csv"
rows = calculate_within_quartiles(metric, tasks)
with open(outfile, 'w') as file:
    writer = csv.writer(file)
    writer.writerows(rows)